In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() # Load environment variables from a .env file if present

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FA162F9DB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FA18C47250>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
ans=model.invoke([HumanMessage(content="Hii, My name is Yash")])
ans

AIMessage(content="Hello Yash! It's nice to meet you. 👋\n\nWhat can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 15, 'total_tokens': 39, 'completion_time': 0.043636364, 'prompt_time': 0.00125204, 'queue_time': 0.24786758, 'total_time': 0.044888404}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b10fa52f-53ae-41cb-9ecf-234bc8513ca1-0', usage_metadata={'input_tokens': 15, 'output_tokens': 24, 'total_tokens': 39})

In [6]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hii, My name is Yash"),
    AIMessage(content="Hello Yash! It's nice to meet you. 👋\n\nWhat can I do for you today?\n"),
    HumanMessage(content="can you tell my name?")
])

AIMessage(content='Of course! Your name is Yash. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 53, 'total_tokens': 77, 'completion_time': 0.043636364, 'prompt_time': 0.00179185, 'queue_time': 0.24786456, 'total_time': 0.045428214}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2b457ecd-d83f-44ed-a1c4-4a69928b51b7-0', usage_metadata={'input_tokens': 53, 'output_tokens': 24, 'total_tokens': 77})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Yash and I am a software developer.")],
    config=config
)

In [12]:
response.content

"Hi Yash!  \n\nIt's great to meet you. Software development is such a dynamic field. What kind of software do you specialize in?  \n\nAre you working on anything particularly interesting at the moment?\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Yash.  😊 \n\nI remember!  How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 160, 'total_tokens': 183, 'completion_time': 0.041818182, 'prompt_time': 0.003795479, 'queue_time': 0.249036051, 'total_time': 0.045613661}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--dc8ca1e4-1d9c-42b9-9fde-0cedb8070c05-0', usage_metadata={'input_tokens': 160, 'output_tokens': 23, 'total_tokens': 183})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 👋 \n\nIs there anything I can help you with today?\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John. 😊  \n\nI remember!  \n\nDo you have any other questions for me?\n'